In [1]:
import numpy as np
import tensorflow as tf
import os

/home/armughan/anaconda2/envs/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/armughan/anaconda2/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#converts the data from chars to int 
def encode_data(datadir):
    data=open(datadir,'r').read()#loading the file character by character in data
    chars = list(set(data))#removing duplicate characters
    vocab_size = len(chars)#counting unique characters
    char_to_idx={ch:ind for ind,ch in enumerate(chars)} #making a dict for conv to int
    idx_to_char={ind:ch for ind,ch in enumerate(chars)}#dict for converting int back to char (used in sampling)
    encoded=np.array([char_to_idx[l] for l in data]) #converted data
    return vocab_size,char_to_idx,idx_to_char,encoded

In [3]:
def get_data_batch(encoded,seq_per_batch,num_steps):
    chars_per_batch=seq_per_batch*num_steps
    num_batches=(encoded.size//chars_per_batch)
#     num_batches-=int(encoded.size==num_batches*chars_per_batch)
    encoded=encoded[:chars_per_batch*num_batches]
    encoded=encoded.reshape((seq_per_batch,-1))
    for i in range(num_batches-1):
        x=encoded[:,i*num_steps:(i+1)*num_steps]
        y=encoded[:,(i*num_steps)+1:((i+1)*num_steps)+1]
        yield x,y

In [4]:
def form_placeholders(batch_size,num_steps):
    x=tf.placeholder(tf.int32,shape=[batch_size,num_steps],name='inputs')
    y=tf.placeholder(tf.int32,shape=[batch_size,num_steps],name='targets')
    keep_prob=tf.placeholder(tf.float32,name="keep_prob")
    return x,y,keep_prob

    

In [5]:
def form_lstm_cell(num_of_layers,layer_size,keep_prob,batch_size):
    lstm_cell=lambda :tf.contrib.rnn.BasicLSTMCell(layer_size)
    dropout_cell=lambda :tf.contrib.rnn.DropoutWrapper(lstm_cell(), output_keep_prob=keep_prob)
    cell=tf.contrib.rnn.MultiRNNCell([dropout_cell() for i in range(num_of_layers)])
    initial_state=cell.zero_state(batch_size,tf.float32)
    return cell,initial_state

In [6]:
def attach_fc_layers(lstm_output,lstmsize,num_neurons):
    concat_outputs=tf.concat(lstm_output,axis=1)
    flat_outputs=tf.reshape(concat_outputs,[-1,lstmsize])
    logits=tf.layers.dense(flat_outputs,num_neurons)
    preds=tf.nn.softmax(logits)
    return preds,logits
    

In [7]:
def attach_loss(logits,y,vocab_size):
    
    y_one_hot=tf.one_hot(y,vocab_size)
    y_reshaped=tf.reshape(y_one_hot,logits.get_shape())
    entropies=tf.nn.softmax_cross_entropy_with_logits(labels=y_reshaped,logits=logits)
    loss=tf.reduce_mean(entropies)
    return loss

In [8]:
def attach_optimizer(loss, learning_rate, grad_clip):
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    
    global_step=tf.Variable(0,trainable=False,name='global_step')
    optimizer = train_op.apply_gradients(zip(grads, tvars),global_step=global_step)
    
    return optimizer,global_step

In [9]:
class charlevelRNN:
    def __init__(self,params):
        sampling=params.get('sampling',False)
        self.seq_per_batch=params.get('seq_per_batch',100)
        self.num_steps=params.get('num_steps',50)
        self.num_of_lstm_layers=params.get('num_of_lstm_layers',1)
        self.lstm_layer_size=params.get('lstm_layer_size',500)
        self.vocab_size=int(params['vocab_size'])
        self.grad_clip=params.get('grad_clip',5)
        self.lr=params.get('lr',0.001)
        logdir=params.get('logdir',None)
#        self.keep_prob=params.get('keep_prob',0.5)
        if sampling:
            self.seq_per_batch=1
            self.num_steps=1
            
        #building new graph
        tf.reset_default_graph()
#         with tf.device('/gpu:0'):
        #making placeholders
        self.x_placeholder,self.y_placeholder,self.keep_prob_placeholder=form_placeholders(self.seq_per_batch,self.num_steps)
        #forming multilayer lstm cells
        cell,self.lstm_state=form_lstm_cell(self.num_of_lstm_layers,self.lstm_layer_size,self.keep_prob_placeholder,self.seq_per_batch)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.x_placeholder,self.vocab_size)

        # Run each sequence step through the RNN with tf.nn.dynamic_rnn 
        lstm_outputs,self.new_lstm_state = tf.nn.dynamic_rnn(cell,x_one_hot,initial_state=self.lstm_state)
#         lstm_outputs,self.new_lstm_state = tf.nn.dynamic_rnn(cell,x_one_hot,dtype=tf.float32)
        #running lstm outputs to the fc layers
        self.preds,self.logits=attach_fc_layers(lstm_outputs,self.lstm_layer_size,self.vocab_size)


        # Loss and optimizer (with gradient clipping)
        self.loss =  attach_loss(self.logits,self.y_placeholder,self.vocab_size)
        self.optimizer,self.step = attach_optimizer(self.loss,self.lr,self.grad_clip)

        loss_summary=tf.summary.scalar('loss',self.loss)
#         self.acc_summary=tf.summary.scalar('accuracy',self.accuracy)
        if not sampling:
            self.summaries=tf.summary.merge_all()
            self.file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())

        self.saver=tf.train.Saver()
        self.initializer=tf.global_variables_initializer()

In [ ]:
num_epochs=50
keep_prob=0.5
seq_per_batch=100
num_steps=150
data_dir='shakespeare_input.txt'
# data_dir='abc.txt'
vocab_size,char_to_idx,idx_to_char,encoded=encode_data(data_dir)
# save_every_n_iter=50
log_every_n_iter=10
# savedir='/output/my_rnn_model.ckpt'
# term_log_file='/output/term_log.txt'
savedir='/my_rnn_model.ckpt'
term_log_file='/term_log.txt'

model_params={
    'sampling':False,
    'seq_per_batch':seq_per_batch,
    'num_steps':num_steps,
    'num_of_lstm_layers':3,
    'lstm_layer_size':550,
    'vocab_size':vocab_size,
#     'logdir':'/output/tf_logs_rnn/run/',
    'logdir':'/tf_logs_rnn/run/',
    'lr':0.01

}
term_log_file=open(term_log_file,'w')

model=charlevelRNN(model_params)


with tf.Session() as sess:
    if os.path.isfile(savedir):
        model.saver.restore(sess,savedir)
    else:
        sess.run(model.initializer)
    lstm_state=sess.run(model.lstm_state)
    for epoch in range(num_epochs):
        if(epoch%10==0):
            model.lr/=10
        print ('----------------------new epoch-----------------------------')
        for x,y in get_data_batch(encoded,seq_per_batch,num_steps):
            print ('epoch no=>',epoch)
            step=sess.run(model.step)
            print ("Step no=> ",step)
            feed_dict={model.x_placeholder:x,model.y_placeholder:y,model.lstm_state:lstm_state,model.keep_prob_placeholder:keep_prob}
            if(step%log_every_n_iter==0):
                _,loss,lstm_state,summary=sess.run([model.optimizer,model.loss,model.new_lstm_state,model.summaries],feed_dict=feed_dict)
                model.file_writer.add_summary(summary,step)
                print ('loss encountered ',loss)
                term_log_file.write('epochno/step '+str(epoch)+'/'+str(step)+'=> loss encountered '+str(loss)+'\n')
                f=open('/output/ep_step.txt','a+')
                f.write(str(epoch)+'/'+str(step)+'\n')
                f.close()
            else:
                _,loss,lstm_state=sess.run([model.optimizer,model.loss,model.new_lstm_state],feed_dict=feed_dict)
        model.saver.save(sess,savedir)
    model.file_writer.close()
    term_log_file.close()

In [12]:
#forming samples
seed='The horse was '

num_samples=1000
keep_prob=1
data_dir='shakespeare_input.txt'
vocab_size,char_to_idx,idx_to_char,encoded=encode_data(data_dir)
print (encoded.size)
sequence=[char_to_idx[seedchar] for seedchar in seed]
# savedir='/output/my_rnn_model.ckpt'
savedir='my_rnn_model.ckpt'
model_params={
    'sampling':True,
    'num_of_lstm_layers':3,
    'lstm_layer_size':550,
    'vocab_size':vocab_size 
}


model=charlevelRNN(model_params)


def select_from_probs(probs,top_n,vocab_size):
    probs=probs.flatten()
    probs[np.argsort(probs)[:-top_n]]=0
    probs=probs/sum(probs)
    return np.random.choice(vocab_size,1,p=probs)


model=charlevelRNN(model_params)
with tf.Session() as sess:
    model.saver.restore(sess,savedir)
    lstm_state=sess.run(model.lstm_state)
    for ci in sequence:#seeding the model
        x=np.array([[ci]])
        feed_dict={model.x_placeholder:x,model.lstm_state:lstm_state,model.keep_prob_placeholder:keep_prob}
        lstm_state=sess.run(model.new_lstm_state,feed_dict=feed_dict)
        
    for i in range(num_samples):
        x=np.array([[sequence[-1]]])
        feed_dict={model.x_placeholder:x,model.lstm_state:lstm_state,model.keep_prob_placeholder:keep_prob}
        probs,lstm_state=sess.run([model.preds,model.new_lstm_state],feed_dict=feed_dict)
        si=select_from_probs(probs,2,vocab_size)
        sequence.append(si[0])

conv_list=[idx_to_char[ind] for ind in sequence ]
print (''.join(conv_list))

4573338
INFO:tensorflow:Restoring parameters from my_rnn_model.ckpt
The horse was HFbAE.wlATH&lAF3lHtclPLlPALvlHtclPLlPALpgclPbHalbATT.:AHblbATlPT&
FtYlbLlAT&lPAHgglITlbATl HBTx..,]rWwj]flVXf;oqM.:ATlbAFtYlFtlbATlPb&HtYT&PElHtclPL3TbF3TPlbATlPbH&PE.]PlbALplPAHgblITlbLlPTTlbAT3lHtclPL3TbF3TP.:AHblbATlPb&HtYTlbLlbAFPlPLtlL lbATlvH&P.wlAH
TlPTtblbLLlPbHtcPlHlPLtElHtclbATlPbHbTE.]tclbATlbAFtYlbATlb&HFbF&PlL lbATlvL&gc.VFgglbATtlbAHblATlAHbAlbLLlPb&HtYTgalITTtE.]tclbAHblAFPlPbHbTlL lbATlvHalbATlPb&HtYTlPb&HtYT&.:AHblbLlATH
TtlbATlPLtlbLlPTTlAF3lbLlbAFPE.:ATlbAFtYlbATlPT&
FBTlL lbATlvL&gclPAHgglIT.]tclbATlb&H
TggT&lL lbATlvL&gclHPlbATa.:AHblbLlAT&lbLlbATlbAFtYlHtclATH
TtlPAHgglPb&HFYAb.:ATlbAFT lbATalATH&lbATlPb&TtYbAlL lbAFPlbAFtYPElbAHb.wlAH
TlITTtlbLlbHOTlbATlbALpPHtclbAFtYE.]tclbAHblbATlPT&
FBTlL lAFPlATHcPlbAHblPALvPE.:AHblbATlbAFtYlbALplAHPblPL3TbAFtYlbATlITH&PlHtclbATlPb&HtYT&E.:ATlPbHbpTlbLlATH&lbATlPbH&PlL lAF3lbAT&T.wtlb&THPLtlbLlbATlbA&LtTElHPlHl3L&TlPb&HFYAb.:LlPTTlbAT3lbATtlHtcl